# Tests módulo `ariaclinprot`

In [1]:
import ariaclinprot as acp

## Prescripciones

In [2]:
pvdf, ccdf, oardf = acp.parse_prescription('../../prescripciones/ORL.csv')

In [3]:
pvdf

,Volume,Dose,FxDose
0,PTV_HR,70.000,2.000
1,PTV_LR,54.000,1.543


In [4]:
ccdf

,Volume,Min,Max,AtLeast,NoMore
0,PTV_HR,,,"[95, 95, 51.3]","[5, 105, 56.7]"
1,PTV_LR,,,"[95, 95, 51.3]","[5, 105, 56.7]"


In [5]:
oardf

,Organ,Dmean,Dmax,DosimPars
0,parótida derecha,26 Gy,,[V30$50 %]
1,parótida izda,26 Gy,,[V30$50 %]
2,médula,,45 Gy,[]
3,tronco cerebral,,,"[D53$100%, V60$30%]"
4,cristalino derecho,,8 Gy,[]
5,cristalino izquierdo,,8 Gy,[]
6,oido interno derecho,,50 Gy,[]
7,oido interno izquierdo,,50 Gy,[]
8,laringe,,70 Gy,[]
9,mandíbula,,70 Gy,[]


## Escritura de protocolos clínicos

In [5]:
bbx = acp.parseProt('BareBone.xml')
TreatmentSite = 'Prostata + vvss hipofx'
acp.modPreview(bbx, ID='ArmaduraTest', TreatmentSite=TreatmentSite)
acp.modPhaseID(bbx, 'Bach')
structureName = 'Recto'
acp.addStructure(bbx, structureName)
acp.addPlanObjetive(bbx, ID=structureName, vParameter=85, vDose=1, vTotalDose=20, vModifier=11)
structureName = 'PTV'
acp.addStructure(bbx, structureName)
acp.addPlanObjetive(bbx, structureName, vParameter=95, vDose=3*.95, vTotalDose=60*.95, vModifier=0, vPrimary='true')
acp.addPlanObjetive(bbx, structureName, vParameter=5, vDose=3*1.07, vTotalDose=60*1.07)
structureName = 'Recto'
acp.addQualityIndex(bbx, ID= structureName, vType=5, vModifier=1, vValue=85, vTypeSpecifier=20, vReportDQPValueInAbsoluteUnits='true')
acp.writeProt(bbx, protout='TestSalida.xml')

In [7]:
bbx = acp.parseProt('BareBone.xml')
TreatmentSite = 'Head and Neck'
acp.modPreview(bbx, ID='ArtificialHN', TreatmentSite=TreatmentSite)
acp.modPhaseID(bbx, 'ORL')
structureName = 'Recto'
acp.addStructure(bbx, structureName)
acp.addPlanObjetive(bbx, ID=structureName, vParameter=85, vDose=1, vTotalDose=20, vModifier=11)
structureName = 'PTV'
acp.addStructure(bbx, structureName)
acp.addPlanObjetive(bbx, structureName, vParameter=95, vDose=3*.95, vTotalDose=60*.95, vModifier=0, vPrimary='true')
acp.addPlanObjetive(bbx, structureName, vParameter=5, vDose=3*1.07, vTotalDose=60*1.07)
structureName = 'Recto'
acp.addQualityIndex(bbx, ID= structureName, vType=5, vModifier=1, vValue=85, vTypeSpecifier=20, vReportDQPValueInAbsoluteUnits='true')

In [21]:
bbx = acp.parseProt('BareBone.xml')
TreatmentSite = 'Head and Neck'
acp.modPreview(bbx, ID='ArtificialHN', TreatmentSite=TreatmentSite)
acp.modPhaseID(bbx, 'ORL')
for pv in pvdf.itertuples():
    acp.addStructure(bbx, structureName=pv.Volume)
for pv in pvdf.itertuples():
    ccVolume = ccdf[ccdf.Volume == pv.Volume]
    if ccVolume.Min.str.match(' +') == False:
        a=1
    if ccVolume.Max.str.match(' +') == False:
        a=1
    if ccVolume.AtLeast.values[0]:
        print(ccVolume.AtLeast)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
if ccdf[ccdf.Volume == Volume].Min.values :
    print('Hola')
else:
    print('Adios')

Hola


In [11]:
ccdf[ccdf.Volume == Volume].Min.values

array(['  '], dtype=object)

In [13]:
ccdf[ccdf.Volume == Volume].Min.str.match(' +')

1    True
Name: Min, dtype: bool

In [16]:
for pv in pvdf.itertuples():
    print(pv.FxDose)

2.000
1.543
